In [1]:
from NumbaMinpack import hybrd, lmdif, minpack_sig
from numba import njit, cfunc
import numpy as np
from scipy.optimize import root

In [2]:
@cfunc(minpack_sig)
def myfunc(x, fvec, args):
    fvec[0] = x[0]**2 - 30.0*x[1]
    fvec[1] = x[1]**2 - 8.0*x[0]

funcptr = myfunc.address 
    
@njit
def myfunc_scipy(x):
    return np.array([x[0]**2 - 30.0*x[1],
                     x[1]**2 - 8.0*x[0]])

In [3]:
x_init = np.array([10.0,10.0])
neqs = 2
args = np.array([0.0])

In [4]:
xsol, fvec, success, info = lmdif(funcptr, x_init, neqs, args)
sol_sp = root(myfunc_scipy,x_init,method='lm')

print('NumbaMinpack (lmdif):',xsol)
print('scipy (lmdif):       ',sol_sp.x)

print()

xsol, fvec, success, info = hybrd(funcptr, x_init, args)
sol_sp = root(myfunc_scipy,x_init,method='hybr')

print('NumbaMinpack (hybrd):',xsol)
print('scipy (hybrd):       ',sol_sp.x)

NumbaMinpack (lmdif): [19.30978769 12.42893002]
scipy (lmdif):        [19.30978769 12.42893002]

NumbaMinpack (hybrd): [19.30978769 12.42893002]
scipy (hybrd):        [19.30978769 12.42893002]


For small problems that take very little time within Minpack, NumbaMinpack will be faster than scipy. This is because scipy sets up the optimization problem in python, which can take more time then the actual optimization. For larger optimization problems, scipy and NumbaMinpack should take about the same amount of time.

In [5]:
%timeit lmdif(funcptr, x_init, neqs, args)
%timeit root(myfunc_scipy,x_init,method='lm')
print()
%timeit hybrd(funcptr, x_init, args)
%timeit root(myfunc_scipy,x_init,method='hybr')

7.29 µs ± 91.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
157 µs ± 2.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

6.11 µs ± 54.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
59.9 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# NumbaMinpack within jit compiled function works
@njit
def test():
    sol = lmdif(funcptr, x_init, neqs, args)
    return sol
test()

(array([19.30978769, 12.42893002]),
 array([5.68434189e-14, 0.00000000e+00]),
 True,
 2)

In [7]:
# scipy within jit compiled function does not work
@njit
def test_sp():
    sol_sp = root(myfunc_scipy,x_init,method='lm')
    return sol_sp
test_sp()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'root':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-7-9e33a68959a9>", line 4:[0m
[1mdef test_sp():
[1m    sol_sp = root(myfunc_scipy,x_init,method='lm')
[0m    [1m^[0m[0m
[0m